# Installasi Darknet - Jalankan sekali saja

## Mounting GDRIVE dan cek cuDNN

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls -la "/content/gdrive/My Drive/darknet"
!/usr/local/cuda/bin/nvcc --version

In [0]:
#bekerja di direktori ini (kalau sudah ada)
%cd /content/gdrive/"My Drive"/darknet

/content/gdrive/My Drive/darknet


## INSTALLASI cuDNN + DARKNET DI GOOGLE DRIVE PRIBADI**

Uncomment instruksi komputer agar di jalankan sekali saja.

apabila sudah terkompilasi akan lebih mudah untuk dijalankan lagi.




In [0]:
#bekerja dalam direktory darknet di google colab
#%cd /content/gdrive/"My Drive"/darknet

## DOWNLOAD FILE INI:
#https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.5.0.56/prod/10.0_20190219/cudnn-10.0-linux-x64-v7.5.0.56.tgz
#buat direktori cuDNN dan letakkan file ini disitu.

# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf /content/gdrive/My\ Drive/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

In [0]:
#UNTUK SEKALI SAJA KERJAKAN INSTALLASI INI. Uncomment sema instruksi dibawah ini

#!git clone https://github.com/kriyeng/darknet/ .
#!ls
#!git checkout feature/google-colab
!make clean
!make
#!chmod +x ./darknet

#CHECK HAsilnya baris akhir harus seperti ini:
#g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCVpkg-config --cflags opencv` -DGPU (...)`


In [0]:
!chmod +x ./darknet
!./darknet

#seharusnya keluar seperti dibawah, berarti sudah bisa di pakai
#usage: ./darknet <function>  

usage: ./darknet <function>


## LOADING HELPER

In [0]:
#download files
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [0]:
!./darknet detect cfg/yolov3.cfg yolov3.weights /content/burung.jpg -dont-show


In [0]:
imShow('predictions.jpg')

## PERSIAPAN DATASET

In [0]:
!cp /content/gdrive/"My Drive"/datatrain.ZIP /content

In [0]:
%cd /content/
!unzip /content/datatrain.ZIP

In [0]:
!cp /content/gdrive/"My Drive"/qut_img.ZIP /content
!unzip /content/qut_img.ZIP

In [0]:
%cd /content/gdrive/"My Drive"/darknet

/content/gdrive/My Drive/darknet


In [0]:
def buildlist(dir='./dataval',filename="file.txt"):
    flist = glob.glob(dir + '/*.jpg')
    flist.sort()
    for idx, i in enumerate(flist):
        fn = i.replace('\\','/')
        flist[idx] = fn
    
    with open(filename,'w') as f:
        for i in flist:
            f.write(i+'\n')

In [0]:
import glob
buildlist(dir="/content/yolov3/datatrain",filename="datatrain.txt")
buildlist(dir="/content/img",filename="img.txt")

In [0]:
!cat datatrain.txt >> files.txt
!cat img.txt >> files.txt

In [0]:
buildlist(dir='/content/yolov3/dataval',filename="dataval.txt")

# Training

In [0]:
## ALTERNATIF 1 - Copy data dari drive anda.

# Copy your compressed file
#!cp -r "/content/gdrive/My Drive/darknet/img/img.tgz" /content/img

# Uncompress zipped file
#!tar -xzvf /content/img/img.tgz

In [0]:
## ALTERNATIF 2 - Copy dari github

# Git clone directly to ./img folder
#!git clone https://[your-repository] ./img

# Check the result - Uncomment when you checked for speed up further runs
#!ls -la ./img

### MULAI TRAINING

In [0]:
!cat "/content/gdrive/My Drive/darknet/obj.data"

classes= 1
train  = /content/gdrive/My\ Drive/darknet/datatrain.txt
valid  = /content/gdrive/My\ Drive/darknet/dataval.txt
names = data/obj.names
backup = backup

In [0]:
!./darknet detector calc_anchors obj.data -num_of_clusters 9 -width 416 -height 416


 num_of_clusters = 9, width = 416, height = 416 
Couldn't open file: /content/gdrive/My Drive/darknet/datatrain.txt


#### TRAINING BIG YOLO

In [0]:
#!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/darknet53.conv.74" -dont_show 

# Mulai training dengan custom weight
#!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/weights/yolov_last.weights" -dont_show 

#### TRAINING TINY YOLO

In [0]:
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3-tiny-obj.cfg" "/content/gdrive/My Drive/darknet/yolov3-tiny.conv.15" -dont_show 

# Mulai training dengan custom weight
#!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/weights/yolov_last.weights" -dont_show 

In [0]:
!cat obj.data

In [0]:
######################## FIXED:ZIP AND UPLOAD FOLDER TO DRIVE ########################
from google.colab import files
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import os
import sys

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.ZIP', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[0:])

In [0]:
zipfolder('/content/ikan_kkp','/content/gdrive/My Drive/darknet/backup/')